In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv("../../trainLabels.csv")

In [33]:
df.iloc[0]['level']

numpy.int64

In [37]:
n = len(df)
samples = {}
i = 0
while i < 1000:
    index = np.random.randint(0, n)
    if df.iloc[index]['image'] not in samples:
        samples[df.iloc[index]['image']] = df.iloc[0]['level']
        i += 1

tests = {}
i = 0
while i < 1000:
    index = np.random.randint(0, n)
    if (df.iloc[index]['image'] not in samples) and (df.iloc[index]['image'] not in tests):
        tests[df.iloc[index]['image']] = df.iloc[0]['level']
        i += 1